In [2]:
import pandas as pd
import numpy as np
from numpy import ma
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.cluster.hierarchy import dendrogram, fcluster
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
np.set_printoptions(precision=2)
pd.set_option('display.precision', 2)

from sklearn.datasets import load_iris
iris_data = load_iris(return_X_y=False, as_frame=True)
iris_data.data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
X = iris_data.data
target_labels = iris_data

# X = iris_data.data.iloc[:10, :]
# target_labels = iris_data.target[:10]

len(X.drop_duplicates())

149

In [5]:
X[X.duplicated()]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
142,5.8,2.7,5.1,1.9


In [23]:
a = np.arange(9).reshape(3, 3)
np.fill_diagonal(a, 10)
a

array([[10,  1,  2],
       [ 3, 10,  5],
       [ 6,  7, 10]])

In [24]:
am = ma.MaskedArray(a)
am[a==10] = ma.masked
~am.mask

array([[False,  True,  True],
       [ True, False,  True],
       [ True,  True, False]])

In [31]:
for i in np.nditer(np.nonzero(~am[1].mask)):
    print(i)

0
2


In [45]:
am.mask[0][0]

True

In [52]:
class AgglomerativeClustering():
    def __init__(self, n_clusters=2, metric='euclidean', linkage='single'):
        self.n_clusters = n_clusters

        metric_choices = {
            'euclidean': lambda a, b: np.sqrt(((a-b) ** 2).sum()),
            'precomputed': None
        }
        try:
            metric_func = metric_choices[metric]
        except KeyError as e:
            raise ValueError(
                f"Unknown metric."
            ) from e
        self._metric_func = metric_func

        def ward_join(p, q, D):
            N_p = np.count_nonzero(self.labels_ == p)
            N_q = np.count_nonzero(self.labels_ == q)
            D_t = np.ndarray((D.shape[0])) # new array
            # D_t.fill(np.inf)
            # for r in index list of existing clusters
            assert D[p][q] == np.min(D)
            for r in np.nditer(np.nonzero(~D.mask[p])):
                N_r = np.count_nonzero(self.labels_ == r)
                assert D[p][r] >= D[p][q]
                print(p, r, D[q][r], p, q, D[p][q])
                assert D[q][r] >= D[p][q]

                assert D[p][r] ** 2 >= D[p][q] ** 2
                assert D[q][r] ** 2 >= D[p][q] ** 2

                assert (N_r + N_p) * D[p][r] ** 2 > N_r * D[p][q]
                assert (N_r + N_q) * D[q][r] ** 2 > N_r * D[p][q]
                
                D_t[r] = np.sqrt( (N_r + N_p) * D[p][r] ** 2 \
                    + (N_r + N_q) * D[q][r] ** 2 \
                    - N_r * D[p][q] ** 2 ) / (N_p + N_q + N_r)
            # print(np.argmin(D_t))
            return(D_t)

        linkage_choices = {
            'single': lambda i, j, D: np.minimum(D[i], D[j]),
            'average': lambda i, j, D: (D[i]+D[j])/2,
            'complete': lambda i, j, D: np.maximum(D[i], D[j]),
            'ward': ward_join
        }
        try:
            join_func = linkage_choices[linkage]
        except KeyError as e:
            raise ValueError(
                f"Unknown linkage. Should be one of {linkage_choices.keys()}"
            ) from e
        if linkage == 'ward' and metric != 'euclidean':
            raise ValueError(
                f"Only euclidean disance is availible for ward's method."
            )
        self._join_func = join_func
        self._linkage = linkage

        self.distance_matrix = None
        self.children_ = None

    def __compute_distance_matrix(self, X):
        N = X.shape[0]
        matrix = np.empty((N, N))
        for i in range(N):
            for j in range(N):
                dist = self._metric_func(X[i], X[j])
                matrix[i,j] = dist
        self.distance_matrix = matrix

    def fit(self, X):
        if type(X) == pd.core.frame.DataFrame:
            X = X.to_numpy()

        if self._metric_func != None:
            self.__compute_distance_matrix(X)
            # if self._linkage == 'ward':
            #     scaler = StandardScaler()
            #     D = scaler.fit_transform(D)
            # self.distance_matrix = D
        else:
            self.distance_matrix = X
        
        D = self.distance_matrix.copy()
        N = D.shape[0]

        agglomerative_schedule = []

        elem_count = np.zeros((N,1), dtype='int')
        clust_map = np.arange(N, dtype=int)

        self.children_ = np.ndarray((N-1, 2), dtype='int')
        self.distances_ = np.ndarray((N-1))
        self.labels_ = np.arange(N)

        mask = np.zeros_like(D)
        D_masked = ma.masked_array(D, mask)
        D_masked.fill_value = np.inf
        np.fill_diagonal(D_masked.mask, 1)

        cluster_num = N
        for k in range(N-1):
            # D_masked[D_masked == 0] = ma.masked
            ind_min_flat = np.argmin(D_masked)
            ind_min = np.unravel_index(ind_min_flat, D_masked.shape)
            i, j = ind_min

            new_cluster = self._join_func(i, j, D_masked)
            elem_count[i] += elem_count[j]
            elem_count[j] = 0

            agglomerative_schedule.append((i+1, j+1,
                D_masked[ind_min].copy(), elem_count[i]))
            self.distances_[k] = D_masked[ind_min]

            self.children_[cluster_num - N, :] = [clust_map[i], clust_map[j]]
            clust_map[i] = cluster_num
            cluster_num += 1

            D_masked[:, [j]] = ma.masked
            D_masked[[j], :] = ma.masked
            D_masked.data[:, [i]] = np.atleast_2d(new_cluster).T
            D_masked.data[[i], :] = new_cluster

            cluster_count = N - k -1
            if cluster_count == self.n_clusters:
                self.labels_ = np.where(self.labels_ == j, i, j)

        self.agglomerative_schedule =  agglomerative_schedule
        
        return self

In [53]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    return linkage_matrix

linkage_types = ['ward', 'complete', 'average', 'single']
cutoff_thresholds = [0.55, 0.5, 0.38, 0.25]
linkage_colors = dict(zip(linkage_types, cutoff_thresholds))
labels = [' Уорда', 'Полной связи', "Средней связи", "Ближнего соседа"]
linkage_labels = dict(zip(linkage_types, labels))

mpl.rcParams.update(mpl.rcParamsDefault)
hclust = dict()
fig, axs = plt.subplots(2, 2)
axs_indices = [(i, j) for i in range(2) for j in range(2)]
for linkage_type, axs_ind in zip(linkage_types, axs_indices):
    model = AgglomerativeClustering(linkage=linkage_type)
    clustering = model.fit(X)
    axs[axs_ind].set_title(linkage_labels[linkage_type])
    axs[axs_ind].axhline(y=linkage_colors[linkage_type], linestyle='dashed', color='black')
    linkage_matrix = plot_dendrogram(
        clustering,
        truncate_mode="level",
        p=6,
        ax=axs[axs_ind],
        labels=None,
        color_threshold = linkage_colors[linkage_type]
        # orientation='right'
        )

    hclust[linkage_type] = {
        "clustering": clustering,
        "linkage_matrix": linkage_matrix
    }
fig.tight_layout()
plt.show()

101 0 4.208325082500163 101 142 0.0
101 1 4.180908992073374 101 142 0.0
101 2 4.334743360338648 101 142 0.0
101 3 4.177319714841084 101 142 0.0
101 4 4.246174749112429 101 142 0.0
101 5 3.9255572852781038 101 142 0.0
101 6 4.263801121065568 101 142 0.0
101 7 4.120679555607302 101 142 0.0
101 8 4.31045241245046 101 142 0.0
101 9 4.143669871020132 101 142 0.0
101 10 4.124318125460256 101 142 0.0
101 11 4.077989700820736 101 142 0.0
101 12 4.244997055358224 101 142 0.0
101 13 4.645427859734773 101 142 0.0
101 14 4.448595283907045 101 142 0.0
101 15 4.255584566190643 101 142 0.0
101 16 4.27668095606862 101 142 0.0
101 17 4.168932717135165 101 142 0.0
101 18 3.9166312055132275 101 142 0.0
101 19 4.149698784249285 101 142 0.0
101 20 3.885871845545089 101 142 0.0
101 21 4.086563348340509 101 142 0.0
101 22 4.685082710048992 101 142 0.0
101 23 3.7907782842049724 101 142 0.0
101 24 3.823610858861032 101 142 0.0
101 25 3.9837168574084174 101 142 0.0
101 26 3.9534794801541584 101 142 0.0
101 27 4

In [ ]:
a = np.arange(149, 2)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
Z=linkage(iris_data.data, 'ward')

In [ ]:
from sklearn.cluster import AgglomerativeClustering as AC
clustering = AC(n_clusters=3, compute_distances=True).fit(iris_data.data)
# print(clustering.distances_.shape)
clustering.labels_

In [ ]:
model = AgglomerativeClustering(linkage='ward')
model.fit(X1.to_numpy())
print(*model.agglomerative_schedule, sep='\n')

In [ ]:
X2 = pd.read_excel('iris_data/clustering.xlsx', sheet_name='dist_matrix', header=None)
X2

In [ ]:
model = AgglomerativeClustering(metric='precomputed')
model.fit(X2.to_numpy())
print(*model.agglomerative_schedule, sep='\n')